# L96 Ginelli Code Redevlop
Redevoloping code to work with observers. Will then allow me to checkpoint with ease.

In [25]:
# Standard Imports

import numpy as np
import time as tm
import pickle
from tqdm.notebook import tqdm
import xarray as xr
import sys
import os

In [26]:
#L96 Ginelli implementation imports

import l96tangent as l96t
import ginelli
from ginelli_observers import *

## Working notes:

- Ginelli forward steps finished. Now need to think about how we calculate FTLEs and do backwards steps.

## Example Observation of TL96

Below we show how to use TL96 integrator and observer in combination

In [29]:
# Make Integrator and observer Objects
runner = l96t.Integrator(K=2, J=2)
looker = l96t.TrajectoryObserver(runner)

# Function for making observations
def make_observations(runner, looker, obs_num, obs_freq, noprog=False):
    """Makes observations.
    runner, integrator object.
    looker, observer object.
    obs_num, how many observations you want.
    obs_freq, adimensional time between observations"""
    for step in tqdm(np.repeat(obs_freq, obs_num), disable=noprog):
        runner.integrate(obs_freq)
        looker.look(runner)
        
make_observations(runner, looker, 20, 0.3)

In [30]:
looker.observations

<xarray.Dataset>
Dimensions:   (time: 20, x: 2, y: 4)
Coordinates:
  * time      (time) float64 0.3 0.6 0.9 1.2 1.5 1.8 ... 4.5 4.8 5.1 5.4 5.7 6.0
  * x         (x) int64 0 1
  * y         (y) int64 0 1 2 3
Data variables:
    X         (time, x) float64 2.724 2.272 3.649 3.823 ... 6.859 6.659 6.64
    Y         (time, y) float64 3.615 0.1142 0.1827 ... -0.1979 -1.264 -2.76
    dx        (time, x) float64 2.724 2.272 3.649 3.823 ... 6.859 6.659 6.64
    dy        (time, y) float64 3.615 0.1142 0.1827 ... -0.1979 -1.264 -2.76
    X_repeat  (time, y) float64 2.724 2.724 2.272 2.272 ... 6.659 6.64 6.64
Attributes:
    h:                    1
    Fs:                   10
    Ff:                   6
    c:                    10
    J:                    2
    K:                    2
    Number of variables:  6
    b:                    4.47213595499958
    dt:                   0.001

## Example Ginelli Algorithm Observations

Below is how you observe the Ginelli algorithm in the new code

In [31]:
runner = l96t.Integrator(K=2, J=2)
ginelli_stepper = ginelli.Forward(runner, 0.1) #relies on runner

In [32]:
# Defining observers

Rlooker = RMatrixObserver(ginelli_stepper)
BLVlooker = BLVMatrixObserver(ginelli_stepper)

In [33]:
def make_observations(runner, lookers, obs_num, obs_freq, noprog=False):
    """Makes observations of Ginelli Algorithm.
    runner, ginelli.Forward object.
    looker, ginelli observer objects.
    obs_num, how many observations you want.
    obs_freq,  time between observations"""
    for step in tqdm(np.repeat(obs_freq, obs_num), disable=noprog):
        runner.run(obs_freq)
        for looker in lookers:
            looker.look(runner)

In [34]:
make_observations(ginelli_stepper, [Rlooker, BLVlooker], 2, 2)

In [35]:
BLVlooker.observations

<xarray.Dataset>
Dimensions:   (le_index: 6, row: 6, time: 2)
Coordinates:
  * time      (time) float64 0.2 0.4
  * le_index  (le_index) int64 1 2 3 4 5 6
Dimensions without coordinates: row
Data variables:
    BLV       (time, row, le_index) float64 -0.05678 -0.03769 ... -0.009038
Attributes:
    h:                    1
    Fs:                   10
    Ff:                   6
    c:                    10
    J:                    2
    K:                    2
    Number of variables:  6
    b:                    4.47213595499958
    dt:                   0.001
    tau:                  0.1